# Comparaison Multi-Stratégies : Fixed vs Adaptive

Ce notebook permet de comparer plusieurs configurations de la stratégie Envelope :
- **Baseline** : Paramètres fixes (configuration actuelle)
- **Regime Adaptive** : Adaptation selon le régime de marché (BULL/BEAR/RECOVERY)
- **Custom** : Vos propres adaptations

Le système génère automatiquement un rapport comparatif avec recommandation.

## 1️⃣ Configuration et chargement des données

In [1]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Backtest engine
from utilities.strategies.envelopeMulti_v2 import EnvelopeMulti_v2
from utilities.data_manager import ExchangeDataManager

# Système adaptatif
from core import calculate_regime_series, DEFAULT_PARAMS
from core.params_adapter import FixedParamsAdapter, RegimeBasedAdapter
from core.backtest_comparator import BacktestComparator

c:\Users\jacks\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuration du backtest
BACKTEST_LEVERAGE = 10
START_DATE = "2023-01-01"
END_DATE = "2024-12-31"

# Paramètres live (à adapter pour backtest)
params_live = {
    "BTC/USDT:USDT": {"size": 0.1, "ma_base_window": 7, "envelopes": [0.07, 0.1, 0.15]},
    "ETH/USDT:USDT": {"size": 0.1, "ma_base_window": 7, "envelopes": [0.07, 0.1, 0.15]},
    "SOL/USDT:USDT": {"size": 0.1, "ma_base_window": 7, "envelopes": [0.07, 0.1, 0.15]},
}

# Conversion pour backtest (diviser size par leverage)
params_coin = {}
for pair, p in params_live.items():
    params_coin[pair] = {
        "ma_base_window": p["ma_base_window"],
        "envelopes": p["envelopes"],
        "src": "close",
        "size": p["size"] / BACKTEST_LEVERAGE  # Ajustement pour V2 engine
    }

print("✅ Configuration chargée")
print(f"   Paires: {list(params_coin.keys())}")
print(f"   Période: {START_DATE} → {END_DATE}")
print(f"   Leverage: {BACKTEST_LEVERAGE}x")

✅ Configuration chargée
   Paires: ['BTC/USDT:USDT', 'ETH/USDT:USDT', 'SOL/USDT:USDT']
   Période: 2023-01-01 → 2024-12-31
   Leverage: 10x


In [3]:
# Chargement des données
exchange = ExchangeDataManager(
    exchange_name="bitget",
    path_download="../database/exchanges"
)

df_list = {}
for pair in params_coin.keys():
    print(f"Chargement {pair}...")
    df = exchange.load_data(pair, "1h", start_date=START_DATE, end_date=END_DATE)
    df_list[pair] = df

oldest_pair = min(df_list, key=lambda p: df_list[p].index.min())
print(f"\n✅ Données chargées. Paire la plus ancienne: {oldest_pair}")

Chargement BTC/USDT:USDT...
Chargement ETH/USDT:USDT...
Chargement SOL/USDT:USDT...

✅ Données chargées. Paire la plus ancienne: BTC/USDT:USDT


In [4]:
# Détection du régime sur BTC (proxy global)
df_btc = exchange.load_data("BTC/USDT:USDT", "1h", start_date=START_DATE, end_date=END_DATE)
regime_series = calculate_regime_series(df_btc, confirm_n=12)

# Distribution des régimes
print("📊 Distribution des régimes détectés:")
regime_counts = regime_series.value_counts(normalize=True) * 100
for regime, pct in regime_counts.items():
    print(f"   {regime.name}: {pct:.1f}%")

KeyError: 'ema50'

## 2️⃣ Définition des stratégies à comparer

In [ ]:
# Liste des adaptateurs à tester
adapters = {
    "Baseline (Fixed)": FixedParamsAdapter(params_coin),
    
    "Regime Adaptive": RegimeBasedAdapter(
        base_params=params_coin,
        regime_series=regime_series,
        regime_params=DEFAULT_PARAMS,
        multipliers={'envelope_std': True},
        base_std=0.10
    ),
    
    # Ajouter d'autres configurations ici...
    # "Conservative": RegimeBasedAdapter(..., base_std=0.08),
    # "Aggressive": RegimeBasedAdapter(..., base_std=0.12),
}

print(f"✅ {len(adapters)} stratégies configurées:")
for name, adapter in adapters.items():
    print(f"   - {name}: {adapter.get_description()}")

## 3️⃣ Exécution des backtests

In [ ]:
# Comparateur
comparator = BacktestComparator(initial_wallet=1000)

# Paramètres communs
backtest_params = {
    "initial_wallet": 1000,
    "leverage": BACKTEST_LEVERAGE,
    "maker_fee": 0.0002,
    "taker_fee": 0.0006,
    "stop_loss": 0.25,
    "reinvest": True,
    "liquidation": True,
    "risk_mode": "scaling",
}

# Exécuter tous les backtests
print("\n🚀 Exécution des backtests...\n")
print("=" * 80)

for strategy_name, adapter in adapters.items():
    print(f"\n▶ {strategy_name}")
    print("-" * 80)
    
    # Initialiser la stratégie
    strategy = EnvelopeMulti_v2(
        df_list=df_list,
        oldest_pair=oldest_pair,
        type=["long", "short"],
        params=params_coin
    )
    
    # Préparer les indicateurs
    strategy.populate_indicators()
    strategy.populate_buy_sell()
    
    # Exécuter avec l'adaptateur
    result = strategy.run_backtest(
        **backtest_params,
        params_adapter=adapter  # 🔑 Adaptation dynamique
    )
    
    # Extraire trades et days du résultat
    df_trades = result['trades']
    df_days = result['days']
    
    # Ajouter au comparateur
    comparator.add_backtest(
        name=strategy_name,
        df_trades=df_trades,
        df_days=df_days,
        metadata={
            "adapter": adapter.get_description(),
            "leverage": BACKTEST_LEVERAGE,
            "period": f"{START_DATE} to {END_DATE}"
        }
    )
    
    # Afficher résultat rapide
    final_wallet = df_days['wallet'].iloc[-1] if len(df_days) > 0 else 1000
    perf = ((final_wallet / 1000) - 1) * 100
    print(f"   Wallet final: ${final_wallet:.2f} ({perf:+.2f}%)")
    print(f"   Trades: {len(df_trades)}")

print("\n" + "=" * 80)
print("✅ Tous les backtests terminés\n")

## 4️⃣ Comparaison et recommandation

In [ ]:
# Générer le rapport comparatif
comparator.print_summary()

In [ ]:
# Voir le classement par métrique spécifique
print("\n📊 Top stratégies par Sharpe Ratio:")
print(comparator.rank('Sharpe Ratio')[['Strategy', 'Sharpe Ratio', 'Total Perf (%)', 'Max DD (%)']].head())

In [ ]:
# Voir le score composite (pondéré)
print("\n🏆 Scores composites (pondération automatique):")
scored = comparator.score()
print(scored[['Strategy', 'Score', 'Total Perf (%)', 'Sharpe Ratio', 'Max DD (%)']].head())

In [ ]:
# Sauvegarder les résultats
output_path = "backtest_comparison_results.csv"
comparator.save_comparison(output_path)
print(f"\n💾 Résultats complets sauvegardés: {output_path}")

## 5️⃣ Analyse détaillée (optionnel)

In [ ]:
# Visualiser l'évolution du wallet pour chaque stratégie
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 6))

for bt in comparator.backtests:
    df_days = bt.df_days.copy()
    if len(df_days) > 0:
        df_days['date'] = pd.to_datetime(df_days['day'])
        plt.plot(df_days['date'], df_days['wallet'], label=bt.name, linewidth=2)

plt.axhline(y=1000, color='gray', linestyle='--', alpha=0.5, label='Initial Wallet')
plt.xlabel('Date')
plt.ylabel('Wallet ($)')
plt.title('Comparaison de l\'évolution du wallet')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 🎯 Conclusion

- Le tableau comparatif montre toutes les métriques clés
- La recommandation est basée sur un score composite pondéré
- Vous pouvez facilement ajouter de nouvelles stratégies dans la section 2️⃣
- Les résultats sont automatiquement sauvegardés en CSV

**Next steps:**
1. Ajuster les pondérations du score selon vos priorités
2. Tester sur différentes périodes
3. Créer des adaptateurs custom avec votre propre logique